# Welcome to ML4CV Assignment
Nicolas Cridlig  
A16002193  
nicolasivan.cridlig@studio.unibo.it  

### Create the virtual enviroment and install required libraries
In a terminal in the main folder, run:
```
python3 -m venv .venv
source .venv/bin/activate
python3 -m pip install -r requirements.txt
```
This only has to be done once. Now, load the jupyter kernel with this virtual enviroment. Note, if you have to modify any libraries in the enviroment and wish to share with others, update the requirements.txt by running

`python3 -m pip freeze > requirements.txt`

in the terminal.

### Libraries

In [ ]:
import numpy as np